This notebook is to get the model predictions for the ships classifier and save it to a csv. The csv includes the embeddings and image URLs. Remember to launch a web server from the data directories

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

2023-07-18 14:38:13.012366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 14:38:13.095387: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-18 14:38:13.516351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/vishesh/anaconda3/lib/:/home/vishesh/anaconda3/lib/python3.9/site-packages/nvidi

In [2]:
model = tf.keras.models.load_model("/home/vishesh/Desktop/datasets/ships-data/Xception-10-0.74.hdf5", compile=False)
model.compile(optimizer=tfa.optimizers.RectifiedAdam(), loss='categorical_crossentropy')

feature_extractor = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('dense_1').output)

2023-07-18 14:38:20.213922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-18 14:38:20.233394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-18 14:38:20.233956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-18 14:38:20.235865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-18 14:38:20.236426: I tensorflow/compiler/xla/stream_executo

In [3]:
# input_data_path = "/home/vishesh/Desktop/datasets/ships-data/X_true_val_240_320/1.npy"
perturbation = "shot_noise"
severity = 10
input_data_path = f"/home/vishesh/Desktop/datasets/ships-data/perturbed_test/varying/{perturbation}/{severity}/1.npy"
ground_truth_path = "/home/vishesh/Desktop/datasets/ships-data/Y_true_val_240_320/1.npy"

input_data = np.load(input_data_path)
ground_truth = np.load(ground_truth_path)

In [4]:
urls = [f"http://localhost:8082/{perturbation}/{severity}/img{i}.png" for i in range(1, input_data.shape[0]+1)]

In [5]:
preds = np.argmax(model.predict(input_data, batch_size=10), axis=1)
features = feature_extractor.predict(input_data, batch_size=10)

2023-07-18 14:38:23.098558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


  6/100 [>.............................] - ETA: 2s

2023-07-18 14:38:24.805614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


100/100 [==============================] - 4s 33ms/step


In [6]:
data_dict = {'url': urls, 'embeddings':features.tolist(), 'predicted': preds, 'actual': ground_truth}
df = pd.DataFrame(data_dict)

df.head(10)

,url,embeddings,predicted,actual
0,http://localhost:8082/shot_noise/10/img1.png,"[0.15127679705619812, 0.0, 0.13076390326023102...",7,7.0
1,http://localhost:8082/shot_noise/10/img2.png,"[0.0, 0.7796505689620972, 0.6098114848136902, ...",8,8.0
2,http://localhost:8082/shot_noise/10/img3.png,"[0.04025774076581001, 0.0, 0.0, 0.089229017496...",5,8.0
3,http://localhost:8082/shot_noise/10/img4.png,"[0.0, 0.04858534038066864, 0.11604125052690506...",4,2.0
4,http://localhost:8082/shot_noise/10/img5.png,"[0.25134122371673584, 0.0, 0.0, 0.0, 0.4035769...",3,8.0
5,http://localhost:8082/shot_noise/10/img6.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0200155898928...",5,0.0
6,http://localhost:8082/shot_noise/10/img7.png,"[0.2322463095188141, 0.2589530348777771, 0.0, ...",1,1.0
7,http://localhost:8082/shot_noise/10/img8.png,"[0.1303481012582779, 0.0, 0.03821878880262375,...",8,8.0
8,http://localhost:8082/shot_noise/10/img9.png,"[0.0, 0.3608892261981964, 0.1379321664571762, ...",4,0.0
9,http://localhost:8082/shot_noise/10/img10.png,"[0.5360773801803589, 0.0, 0.0, 0.0, 0.70785599...",3,3.0


In [7]:
df.to_csv(f"/home/vishesh/Desktop/datasets/ships-data/{perturbation}_{severity}_val.csv")  

In [ ]:
perturbation = "shot_noise"
for severity in range(1,11):
    input_data_path = f"/home/vishesh/Desktop/datasets/ships-data/perturbed_test/varying/{perturbation}/{severity}/1.npy"
    ground_truth_path = "/home/vishesh/Desktop/datasets/ships-data/Y_true_val_240_320/1.npy"

    input_data = np.load(input_data_path)
    ground_truth = np.load(ground_truth_path)

    preds = np.argmax(model.predict(input_data, batch_size=10), axis=1)
    acc = preds[preds == ground_truth].shape[0]/ ground_truth.shape[0]
    print(f"Acc for sev {severity} is {acc}")